# Day 22

Use NFL play-by-play data from 1999 to create a dataset at the game-team level where I can see the score and the number of different types of scoring plays each team had that made up their final score. Ultimately I want to compare the percentage of score by offense/defense to see how teams perform and if there are any outliers at the weekly, team, or season level.

Today's update:
- Updated the play-by-play query for the initial data pull as well as minor data cleaning
- Was able to check each team's score for each game against the schedules table. **Scores match 100%** so my manual calculation is correct.

Current challenges:
- Need to do a sanity check on the frequency of different scoring plays to make sure that I didn't count an offensive TD as a defensive TD or something similar

Solutions:
- Run histograms and utilize frequencies
- Check a few sample cases of rare scoring plays and see if my code places them in the right category

In [192]:
import pandas as pd
import sqlite3
import nfl_data_py as nfl

# Create database connection
conn = sqlite3.connect('../../data/db/database.db')

## Get Score Summary Table

In [193]:
query = """
WITH offense AS (
    SELECT
        game_id,
        season,
        week,
        home_team,
        away_team,
        posteam,
        SUM(touchdown) AS tot_tds,
        SUM(pass_touchdown) AS tot_pass_tds,
        SUM(rush_touchdown) AS tot_rush_tds,
        SUM(return_touchdown) AS tot_ret_tds
    FROM pbp
    WHERE posteam IS NOT NULL 
        AND posteam <> ""
        AND posteam = td_team
        AND (
            pass_touchdown = 1
            OR rush_touchdown = 1
            OR return_touchdown = 1
        )
    GROUP BY game_id, posteam), 
extra_pts AS (
    SELECT
        game_id,
        posteam,
        COUNT(*) AS tot_extra_pts
    FROM pbp
    WHERE extra_point_attempt = 1 AND extra_point_result = 'good'
    GROUP BY game_id, posteam),
field_goals AS (
    SELECT 
        game_id,
        -- Correct for data error in game_id: 2000_11_OAK_DEN
        CASE WHEN game_id = '2000_11_OAK_DEN' 
            AND desc LIKE '%J.Elam%' 
            AND desc LIKE '%field goal%' 
            THEN 'DEN'
        ELSE posteam
        END AS team,
        COUNT(*) AS tot_fgs
    FROM pbp
    WHERE field_goal_attempt = 1 
        AND field_goal_result = 'made'
        AND desc LIKE '%field goal%'
    GROUP BY game_id, team), 
two_pt_convs AS (
    SELECT 
        game_id,
        posteam,
        COUNT(*) AS tot_2pt_conv
    FROM pbp
    WHERE two_point_attempt = 1 AND two_point_conv_result = 'success'
    GROUP BY game_id, posteam),
-- Counts defensive TDs and punt/kickoff return TDs
defense AS (
    SELECT
        game_id,
        td_team AS team,
        COUNT(*) AS tot_def_tds
    FROM pbp
    WHERE touchdown = 1
        AND (
            defteam_score_post <> defteam_score
            OR (defteam_score IS NULL AND defteam_score_post >= 6)
        )
    GROUP BY game_id, td_team),
safeties AS (
    SELECT
        game_id,
        CASE WHEN defteam_score_post <> defteam_score THEN defteam
        ELSE posteam
        END AS team,
        COUNT(*) AS tot_safeties
    FROM pbp
    WHERE safety = 1
    GROUP BY game_id, team),
def_2pt_att AS (
    SELECT
        game_id,
        defteam AS team,
        COUNT(*) AS tot_def_2pt
    FROM pbp
    WHERE desc LIKE '%DEFENSIVE TWO-POINT ATTEMPT%'
        AND defteam_score_post <> defteam_score
    GROUP BY game_id, team),
off_fumb_recovery AS (
    SELECT
        game_id,
        posteam AS team,
        COUNT(*) AS tot_off_fumble_recov_td
    FROM pbp
    WHERE desc LIKE '%fumble%'
        AND posteam_score_post <> posteam_score
        AND touchdown = 1
        AND pass_touchdown = 0
        AND rush_touchdown = 0
        AND return_touchdown = 0
    GROUP BY game_id, team),
joined AS (
    SELECT 
        offense.*,
        CASE WHEN tot_extra_pts IS NULL THEN 0
        ELSE tot_extra_pts
        END AS tot_extra_pts,
        CASE WHEN tot_fgs IS NULL THEN 0
        ELSE tot_fgs
        END AS tot_fgs,
        CASE WHEN tot_2pt_conv IS NULL THEN 0
        ELSE tot_2pt_conv
        END AS tot_2pt_conv,
        CASE WHEN tot_def_tds IS NULL THEN 0
        ELSE tot_def_tds
        END AS tot_def_tds,
        CASE WHEN tot_safeties IS NULL THEN 0
        ELSE tot_safeties
        END AS tot_safeties,
        CASE WHEN tot_def_2pt IS NULL THEN 0
        ELSE tot_def_2pt
        END AS tot_def_2pt,
        CASE WHEN tot_off_fumble_recov_td IS NULL THEN 0
        ELSE tot_off_fumble_recov_td
        END AS tot_off_fumble_recov_td
    FROM offense
    LEFT JOIN extra_pts
        ON extra_pts.game_id = offense.game_id
            AND extra_pts.posteam = offense.posteam
    LEFT JOIN field_goals
        ON field_goals.game_id = offense.game_id
            AND field_goals.team = offense.posteam
    LEFT JOIN two_pt_convs
        ON two_pt_convs.game_id = offense.game_id
            AND two_pt_convs.posteam = offense.posteam
    LEFT JOIN defense
        ON defense.game_id = offense.game_id
            AND defense.team = offense.posteam
    LEFT JOIN safeties
        ON safeties.game_id = offense.game_id
            AND safeties.team = offense.posteam
    LEFT JOIN def_2pt_att
        ON def_2pt_att.game_id = offense.game_id
            AND def_2pt_att.team = offense.posteam
    LEFT JOIN off_fumb_recovery
        ON off_fumb_recovery.game_id = offense.game_id
            AND off_fumb_recovery.team = offense.posteam
)
SELECT *,
    (tot_pass_tds * 6
    + tot_rush_tds * 6
    + tot_ret_tds * 6
    + tot_extra_pts * 1
    + tot_fgs * 3
    + tot_2pt_conv * 2
    + tot_def_tds * 6
    + tot_safeties * 2
    + tot_def_2pt * 2
    + tot_off_fumble_recov_td * 6) AS score,
    -- Use old team abbrev. that matches game_id for teams that moved (pbp data has new names)
    CASE
        WHEN game_id LIKE '%OAK%' AND posteam = 'LV' THEN 'OAK'
        WHEN game_id LIKE '%SD%' AND posteam = 'LAC' THEN 'SD'
        WHEN game_id LIKE '%STL%' AND posteam = 'LA' THEN 'STL'
        ELSE posteam
    END AS team_fixed
FROM joined
"""

df_pbp = pd.read_sql(query, conn)
df_pbp.head(10)

,game_id,season,week,home_team,away_team,posteam,tot_tds,tot_pass_tds,tot_rush_tds,tot_ret_tds,tot_extra_pts,tot_fgs,tot_2pt_conv,tot_def_tds,tot_safeties,tot_def_2pt,tot_off_fumble_recov_td,score,team_fixed
0,1999_01_ARI_PHI,1999,1,PHI,ARI,ARI,2.0,1.0,1.0,0.0,1,4,0,0,0,0,0,25.0,ARI
1,1999_01_ARI_PHI,1999,1,PHI,ARI,PHI,3.0,2.0,1.0,0.0,3,1,0,0,0,0,0,24.0,PHI
2,1999_01_BUF_IND,1999,1,IND,BUF,BUF,1.0,1.0,0.0,0.0,0,2,1,0,0,0,0,14.0,BUF
3,1999_01_BUF_IND,1999,1,IND,BUF,IND,3.0,2.0,1.0,0.0,4,1,0,1,0,0,0,31.0,IND
4,1999_01_CAR_NO,1999,1,NO,CAR,CAR,1.0,1.0,0.0,0.0,1,1,0,0,0,0,0,10.0,CAR
5,1999_01_CAR_NO,1999,1,NO,CAR,NO,1.0,1.0,0.0,0.0,1,2,0,1,0,0,0,19.0,NO
6,1999_01_CIN_TEN,1999,1,TEN,CIN,CIN,4.0,2.0,2.0,0.0,1,2,2,0,0,0,0,35.0,CIN
7,1999_01_CIN_TEN,1999,1,TEN,CIN,TEN,4.0,3.0,1.0,0.0,4,2,0,0,1,0,0,36.0,TEN
8,1999_01_DAL_WAS,1999,1,WAS,DAL,DAL,6.0,5.0,1.0,0.0,5,0,0,0,0,0,0,41.0,DAL
9,1999_01_DAL_WAS,1999,1,WAS,DAL,WAS,4.0,2.0,2.0,0.0,3,2,1,0,0,0,0,35.0,WAS


In [194]:
# Get each team's score from each game from the schedules table in order to check my manual score calculation using play-by-play data
query = """
WITH home_games AS (
    SELECT
        game_id,
        season,
        week,
        home_team AS team,
        home_score AS score
    FROM schedules
    --WHERE week <= 9
    --    AND game_type = 'REG'
), away_games AS (
    SELECT
        game_id,
        season,
        week,
        away_team AS team,
        away_score AS score
    FROM schedules
    --WHERE week <= 9
    --    AND game_type = 'REG'
), stacked AS (
    SELECT *
    FROM home_games
    UNION ALL
    SELECT *
    FROM away_games
)
SELECT
    game_id,
    team,
    score
FROM stacked
"""

df_schedules = pd.read_sql(query, conn)
df_schedules.head(20)

,game_id,team,score
0,1999_01_MIN_ATL,ATL,14.0
1,1999_01_KC_CHI,CHI,20.0
2,1999_01_PIT_CLE,CLE,0.0
3,1999_01_OAK_GB,GB,28.0
4,1999_01_BUF_IND,IND,31.0
5,1999_01_SF_JAX,JAX,41.0
6,1999_01_CAR_NO,NO,19.0
7,1999_01_NE_NYJ,NYJ,28.0
8,1999_01_ARI_PHI,PHI,24.0
9,1999_01_DET_SEA,SEA,20.0


In [195]:
# Join the two tables
merged = df_pbp.merge(
    df_schedules,
    'left',
    left_on=['game_id', 'team_fixed'],
    right_on=['game_id', 'team']
)

merged.head()

,game_id,season,week,home_team,away_team,posteam,tot_tds,tot_pass_tds,tot_rush_tds,tot_ret_tds,...,tot_fgs,tot_2pt_conv,tot_def_tds,tot_safeties,tot_def_2pt,tot_off_fumble_recov_td,score_x,team_fixed,team,score_y
0,1999_01_ARI_PHI,1999,1,PHI,ARI,ARI,2.0,1.0,1.0,0.0,...,4,0,0,0,0,0,25.0,ARI,ARI,25.0
1,1999_01_ARI_PHI,1999,1,PHI,ARI,PHI,3.0,2.0,1.0,0.0,...,1,0,0,0,0,0,24.0,PHI,PHI,24.0
2,1999_01_BUF_IND,1999,1,IND,BUF,BUF,1.0,1.0,0.0,0.0,...,2,1,0,0,0,0,14.0,BUF,BUF,14.0
3,1999_01_BUF_IND,1999,1,IND,BUF,IND,3.0,2.0,1.0,0.0,...,1,0,1,0,0,0,31.0,IND,IND,31.0
4,1999_01_CAR_NO,1999,1,NO,CAR,CAR,1.0,1.0,0.0,0.0,...,1,0,0,0,0,0,10.0,CAR,CAR,10.0


In [196]:
# Check for mismatched scores
(merged['score_x'] - merged['score_y']).sum()

0.0

In [197]:
merged.columns

Index(['game_id', 'season', 'week', 'home_team', 'away_team', 'posteam',
       'tot_tds', 'tot_pass_tds', 'tot_rush_tds', 'tot_ret_tds',
       'tot_extra_pts', 'tot_fgs', 'tot_2pt_conv', 'tot_def_tds',
       'tot_safeties', 'tot_def_2pt', 'tot_off_fumble_recov_td', 'score_x',
       'team_fixed', 'team', 'score_y'],
      dtype='object')

In [198]:
# Clean dataset
merged_clean = merged[['game_id', 'season', 'week', 'team',
       'score_x', 'tot_pass_tds', 'tot_rush_tds', 'tot_ret_tds',
       'tot_extra_pts', 'tot_fgs', 'tot_2pt_conv', 'tot_def_tds',
       'tot_safeties', 'tot_def_2pt', 'tot_off_fumble_recov_td']]

merged_clean = merged_clean.rename(columns={'score_x':'score'})
merged_clean.head()

,game_id,season,week,team,score,tot_pass_tds,tot_rush_tds,tot_ret_tds,tot_extra_pts,tot_fgs,tot_2pt_conv,tot_def_tds,tot_safeties,tot_def_2pt,tot_off_fumble_recov_td
0,1999_01_ARI_PHI,1999,1,ARI,25.0,1.0,1.0,0.0,1,4,0,0,0,0,0
1,1999_01_ARI_PHI,1999,1,PHI,24.0,2.0,1.0,0.0,3,1,0,0,0,0,0
2,1999_01_BUF_IND,1999,1,BUF,14.0,1.0,0.0,0.0,0,2,1,0,0,0,0
3,1999_01_BUF_IND,1999,1,IND,31.0,2.0,1.0,0.0,4,1,0,1,0,0,0
4,1999_01_CAR_NO,1999,1,CAR,10.0,1.0,0.0,0.0,1,1,0,0,0,0,0
